# cancer_maintrain

> Please add a description. Don't forgot to edit the below cell!!

In [ ]:
#| default_exp cancer_maintrain

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive')
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
#| export
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *

ModuleNotFoundError: No module named 'cancer_proj'

For now just get the data however it works. Will fix this later. But want to get `main_train` working for now

## Load the data

In [ ]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir
    


#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [ ]:
#| hide


aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

In [ ]:
#| hide
show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [ ]:
#| hide

show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

## TODO: Can probably put this in `dataloading`. No need to clog this notebook up.

In [ ]:
#| hide

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## main training api

In [ ]:
#| hide

import itertools
initialweights = ['no_pretrain','supervised_pretrain']
fitpolicy = ['fit_one_cycle','fit']

#cartesian product of hps
MyTuple = namedtuple('MyTuple', 'initialweights fitpolicy')
HPs = [MyTuple(x,y) for x,y in list(itertools.product(initialweights, fitpolicy))]

for hp in HPs:
    print(hp.initialweights)

In [ ]:
#| export

class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 fit_policy, #policy for fitting BT
                 tune_fit_policy,
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
                 store_attr()
                 self.vocab = self.dls_valid.vocab


                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_policy,epochs):

        if fit_policy == 'fit':
            learn.fit(epochs)
        
        elif fit_policy == 'fit_one_cycle':
            learn.fit_one_cycle(epochs)
       
        elif fit_policy == 'fine_tune':
            learn.fine_tune(epochs)

        else: raise Exception('Fit policy not of expected form')


    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorith, according to fit_policy"


        bt_model,encoder = create_model(which_model=self.initial_weights,device=device) #create encoder and bt model


        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model, cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])

            main_train.fit(learn,self.fit_policy,epochs=self.num_epochs)

            #torch.save(encoder.state_dict(), self.encoder_path)

        self.encoder = bt_model.encoder


    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        model = LinearModel(encoder=self.encoder,indim=self.indim,outdim=self.outdim) #create 'linear' model (encoder + linear head)
        
        learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_policy=self.tune_fit_policy,epochs=self.numfit) #fine tuning (don't confuse this with fit policy!)

        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)

        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        metrics['acc'] = acc
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['auc_dict'] = auc_dict

        metrics['scores'] = scores
        
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics

if __name__ == '__main__' and on_colab:


    initialweights = ['bt_pretrain','supervised_pretrain']
    fitpolicy = ['fit_one_cycle','fit']

    #cartesian product of hps
    MyTuple = namedtuple('MyTuple', 'initialweights fitpolicy')
    HPs = [MyTuple(x,y) for x,y in list(itertools.product(initialweights, fitpolicy))]


    for _ in range(1):

        d={}
        for initial_weights in initialweights:
            metrics = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights='supervised_pretrain',pretrain=True,fit_policy='fit',tune_fit_policy='fine_tune',num_epochs=20,numfit=20, 
                )()

            d[initial_weights] = {}

            d[initial_weights]['acc'] = metrics['acc']

            d[initial_weights]['scores'] = metrics['scores']

            print(f"With initial_weights={initial_weights}, acc is {d[initial_weights]['acc']}")


        ypred,acc = predict_ensemble(yval,scores1=d['bt_pretrain']['scores'],scores2=d['supervised_pretrain']['scores'])

        print(f'Acc of ensemble is {acc}')




    num=3
    mean_acc = 0

    for _ in range(num):
        metrics = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                    aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                    initial_weights='supervised_pretrain',pretrain=True,fit_policy='fit',tune_fit_policy='fine_tune',num_epochs=20,numfit=20, 
                    )()

        acc = metrics['acc']
        print(acc)
        mean_acc += acc

    print(mean_acc/num)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()